# Rough project/code architecture

1. Load MNIST into some kind of native dataformat (i.e. not with a oneliner that gives you a pytorch dataset)
1. Load entire MNIST into a dataset, then make that into a dataloader
1. Train MNIST with pytorch.  make a note of loss and accuracy. You’ll need to choose an architecture, batch size and number of epochs to train for. 
    * Get architecture from pytorch docs. Should be simple. 
    * Number of epochs is “until results get as good as they are going to get”, i.e. by inspection.
    * Batch size I got no idea. 1 is probably fine, but feel free to try others.
1. Split MNIST into N groups randomly. Create N datasets. Train with federated.py. You should get roughly the final loss and accuracy you saw previously. 
    * Use the same architecture as before.
    * Choose number of rounds same way you chose number of epochs in previous step, i.e. until it converges.
    * Number of epochs per round. Maybe try 1, 5 and 10? Go with 1 unless you get very different results.
    * Batch size should probably be the same as before.
1. Repeat, but split MNIST into N groups with the deck stacked (possibly as extremely as 10 groups each containing data from only one class).

### Snippets

#### Visualize an array
See cell 109 here: https://github.com/williamsmj/pytorch-notes/blob/master/pytorch-60-minute-blitz.ipynb

#### Change native data to dataset, change dataset to native data, and loop over dataset doing something other than training
See https://github.com/williamsmj/pytorch-notes/blob/master/dataset_manipulation.ipynb 


In [2]:
data_path = './MNIST-data/raw'

# location of data and labels
test_labels_file = data_path + '/' + 't10k-labels-idx1-ubyte'
test_data_file = data_path + '/' + 't10k-images-idx3-ubyte'
train_labels_file = data_path + '/' + 'train-labels-idx1-ubyte'
train_data_file = data_path + '/' + 'train-images-idx3-ubyte'

In [3]:
import torch
import torchvision.datasets as dsets

import torchvision.transforms as transforms
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
trans = transforms.Compose([transforms.ToTensor()])

# pytorch datasets that download MNIST set as needed; used only to download files
train_dset = dsets.MNIST(root=data_path, download=False, train=True, transform=trans)
test_dset = dsets.MNIST(root=data_path, download=False, train=False, transform=trans)

#print("Training dset:", train_dset)
#print("Test dset:", test_dset)

In [4]:
torch.__version__

'0.4.1'

In [16]:
# Load MNIST data into numpy arrays
import numpy as np
import struct

def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)
    
# read data and label files into numpy arrays
test_data = read_idx(test_data_file)
test_labels = read_idx(test_labels_file)
train_data = read_idx(train_data_file)
train_labels = read_idx(train_labels_file)

In [4]:
# save CSV files if needed

"""
np.savetxt("./MNIST-data/raw/test_data.csv", test_data.reshape(len(test_data), 784), delimiter=',')
np.savetxt("./MNIST-data/raw/test_labels.csv", test_labels, delimiter=',')
np.savetxt("./MNIST-data/raw/train_data.csv", train_data.reshape(len(train_data), 784), delimiter=',')
np.savetxt("./MNIST-data/raw/train_labels.csv", train_labels, delimiter=',')

"""

# to restore
#np.loadtext()
#reshape((len(), 28, 28))

'\nnp.savetxt("./MNIST-data/raw/test_data.csv", test_data.reshape(len(test_data), 784), delimiter=\',\')\nnp.savetxt("./MNIST-data/raw/test_labels.csv", test_labels, delimiter=\',\')\nnp.savetxt("./MNIST-data/raw/train_data.csv", train_data.reshape(len(train_data), 784), delimiter=\',\')\nnp.savetxt("./MNIST-data/raw/train_labels.csv", train_labels, delimiter=\',\')\n\n'

In [5]:
# Take a look at the data in the arrays

"""
# pick a random sample from the sets of observations
import random
test_sample_num = random.randint(0, len(test_labels))
train_sample_num = random.randint(0, len(train_labels))

# print some arrays to confirm we have data
print("Test labels:", test_labels)
print("Test data:", test_data[test_sample_num])
print("Training labels:", train_labels)
print("Training data:", train_data[train_sample_num])
"""

'\n# pick a random sample from the sets of observations\nimport random\ntest_sample_num = random.randint(0, len(test_labels))\ntrain_sample_num = random.randint(0, len(train_labels))\n\n# print some arrays to confirm we have data\nprint("Test labels:", test_labels)\nprint("Test data:", test_data[test_sample_num])\nprint("Training labels:", train_labels)\nprint("Training data:", train_data[train_sample_num])\n'

In [5]:
# visualize some test samples

"""
import matplotlib.pyplot as plt
%matplotlib inline

img = test_data[test_sample_num]
label = test_labels[test_sample_num]
print('Test sample index: ', test_sample_num)
print('Sample value: ', label)
plt.imshow(img)
"""

NameError: name 'test_data' is not defined

In [7]:
# how is the test data distributed?

"""
print('Test sample histogram:')
plt.hist(test_labels, bins=10)
"""

"\nprint('Test sample histogram:')\nplt.hist(test_labels, bins=10)\n"

In [8]:
# visualize a training sample

"""
img = train_data[train_sample_num]
label = train_labels[train_sample_num]
print('Training sample index: ', train_sample_num)
print('Sample value: ', label)
plt.imshow(img)
"""

"\nimg = train_data[train_sample_num]\nlabel = train_labels[train_sample_num]\nprint('Training sample index: ', train_sample_num)\nprint('Sample value: ', label)\nplt.imshow(img)\n"

In [9]:
# how is the training data distributed?

"""
print('Training data histogram:')
plt.hist(train_labels, bins=10)
"""

"\nprint('Training data histogram:')\nplt.hist(train_labels, bins=10)\n"

In [17]:
# load numpy arrays into pytorch Datasets
import torch
from torch.utils.data import TensorDataset

# create tensors from np arrays
test_data_tensor = torch.from_numpy(test_data)
test_labels_tensor = torch.from_numpy(test_labels)
train_data_tensor = torch.from_numpy(train_data)
train_labels_tensor = torch.from_numpy(train_labels)


# TODO: TRANSFORM THE DATA WHEN THE SETS ARE CREATED
"""
import torch
import torchvision.datasets as dsets

import torchvision.transforms as transforms
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
trans = transforms.Compose([transforms.ToTensor()])
"""

# load tensors into datasets
test_dset = TensorDataset(test_data_tensor, test_labels_tensor, transform=trans)
train_dset = TensorDataset(train_data_tensor, train_labels_tensor, transform=trans)

print("test_dset:",
      test_dset,
      len(test_dset), test_dset[0][0].size(), type(test_dset[0][0])
     )


TypeError: __init__() got an unexpected keyword argument 'transform'

In [3]:
from torch.utils.data import DataLoader

# create dataloaders
batch_size = 1
test_dloader = DataLoader(test_dset, batch_size=batch_size, shuffle=False)
train_dloader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)

print("test_dloader", len(test_dloader),
     )

test_dloader 10000


In [4]:
# set up network

import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## network
class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 10)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def name(self):
        return "MLP"

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def name(self):
        return "LeNet"

In [1]:
import torch.nn.functional as F
from torch.autograd import Variable

model = MLPNet()

learning_rate = 0.0003
momentum = 0.5
num_epochs = 2

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
criterion = nn.CrossEntropyLoss()

#print(enumerate(train_dloader))

for epoch in range(num_epochs):
    avg_loss = 0

    # train
    for batch_idx, (x, target) in enumerate(train_dloader):
        optimizer.zero_grad()
        
        #if (batch_idx+1) % 100 == 0 or (batch_idx+1) == len(train_dloader):
        #    print("batch_idx:", batch_idx)
        #print("shape of x:", x.shape, type(x))
        #print(x[0])
        #print("shape of target:", target.shape, type(target))
        
        # TODO try without the next line
        #x, target = Variable(x), Variable(target)
        
        out = model(x)
        loss = criterion(out, target)
        avg_loss = avg_loss * 0.99 + loss.item() * 0.01
        loss.backward()
        optimizer.step()
        if (batch_idx+1) % 500 == 0 or (batch_idx+1) == len(train_dloader):
            print ('==>>> epoch: {}, batch index: {}, train loss: {:.6f}'.format(
                epoch, batch_idx+1, avg_loss))

            
            
# ADD PLOTTING

# ADD TESTING AFTER TRAINING

NameError: name 'MLPNet' is not defined

In [ ]:
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

## training
model = LeNet()


optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

criterion = nn.CrossEntropyLoss()

for epoch in range(1):
    # training
    ave_loss = 0
    for batch_idx, (x, target) in enumerate(train_dloader):
        optimizer.zero_grad()
        x, target = Variable(x), Variable(target)
        out = model(x)
        loss = criterion(out, target)
        ave_loss = ave_loss * 0.9 + loss.item() * 0.1
        loss.backward()
        optimizer.step()
        if (batch_idx+1) % 100 == 0 or (batch_idx+1) == len(train_dloader):
            print ('==>>> epoch: {}, batch index: {}, train loss: {:.6f}'.format(
                epoch, batch_idx+1, ave_loss))
    # testing
    correct_cnt, ave_loss = 0, 0
    total_cnt = 0
    for batch_idx, (x, target) in enumerate(test_dloader):
        with torch.no_grad():
            x, target = Variable(x), Variable(target)
        out = model(x)
        loss = criterion(out, target)
        _, pred_label = torch.max(out.data, 1)
        total_cnt += x.data.size()[0]
        correct_cnt += (pred_label == target.data).sum()
        # smooth average
        #ave_loss = ave_loss * 0.9 + loss.item() * 0.1
        
        if(batch_idx+1) % 100 == 0 or (batch_idx+1) == len(test_dloader):
            print ('==>>> epoch: {}, batch index: {}, test loss: {:.6f}, acc: {:.3f}'.format(
                epoch, batch_idx+1, ave_loss, correct_cnt * 1.0 / total_cnt))

==>>> epoch: 0, batch index: 100, train loss: 2.298744
==>>> epoch: 0, batch index: 200, train loss: 1.527318
==>>> epoch: 0, batch index: 300, train loss: 1.569507
==>>> epoch: 0, batch index: 400, train loss: 1.890597
==>>> epoch: 0, batch index: 500, train loss: 2.413005
==>>> epoch: 0, batch index: 600, train loss: 1.716513
==>>> epoch: 0, batch index: 700, train loss: 1.455534
==>>> epoch: 0, batch index: 800, train loss: 2.105138
==>>> epoch: 0, batch index: 900, train loss: 2.461570
==>>> epoch: 0, batch index: 1000, train loss: 1.135260
==>>> epoch: 0, batch index: 1100, train loss: 1.655213
==>>> epoch: 0, batch index: 1200, train loss: 1.911636
==>>> epoch: 0, batch index: 1300, train loss: 1.575496
==>>> epoch: 0, batch index: 1400, train loss: 1.406858
==>>> epoch: 0, batch index: 1500, train loss: 1.582742
==>>> epoch: 0, batch index: 1600, train loss: 0.618945
==>>> epoch: 0, batch index: 1700, train loss: 1.205498
==>>> epoch: 0, batch index: 1800, train loss: 2.589466
=

==>>> epoch: 0, batch index: 14700, train loss: 2.361027
==>>> epoch: 0, batch index: 14800, train loss: 2.283446
==>>> epoch: 0, batch index: 14900, train loss: 2.387924
==>>> epoch: 0, batch index: 15000, train loss: 2.339913
==>>> epoch: 0, batch index: 15100, train loss: 2.340443
==>>> epoch: 0, batch index: 15200, train loss: 2.395314
==>>> epoch: 0, batch index: 15300, train loss: 2.316582
==>>> epoch: 0, batch index: 15400, train loss: 2.374598
==>>> epoch: 0, batch index: 15500, train loss: 2.311883
==>>> epoch: 0, batch index: 15600, train loss: 2.336558
==>>> epoch: 0, batch index: 15700, train loss: 2.315594
==>>> epoch: 0, batch index: 15800, train loss: 2.242960
==>>> epoch: 0, batch index: 15900, train loss: 2.224170
==>>> epoch: 0, batch index: 16000, train loss: 2.329422
==>>> epoch: 0, batch index: 16100, train loss: 2.325356
==>>> epoch: 0, batch index: 16200, train loss: 2.314312
==>>> epoch: 0, batch index: 16300, train loss: 2.248769
==>>> epoch: 0, batch index: 16

==>>> epoch: 0, batch index: 29100, train loss: 2.276406
